In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
import pandas as pd
import numpy as np
from time import time
from sklearn.datasets import load_digits

In [ ]:
data = pd.read_csv("datasets/income.csv")
data

In [ ]:
data["education"] = data["education"].replace(
    {
        'Preschool': 'dropout',
        '10th': 'dropout',
        '11th': 'dropout',
        '12th': 'dropout',
        '1st-4th': 'dropout',
        '5th-6th': 'dropout',
        '7th-8th': 'dropout',
        '9th': 'dropout',
        'HS-Grad': 'HighGrad',
        'HS-grad': 'HighGrad',
        'Some-college': 'CommunityCollege',
        'Assoc-acdm': 'CommunityCollege',
        'Assoc-voc': 'CommunityCollege',
        'Bachelors': 'Bachelors',
        'Masters': 'Masters',
        'Prof-school': 'Masters',
        'Doctorate': 'Doctorate'
    }
)
data["marital-status"] = data["marital-status"].replace(
    {
        'Never-married': 'NotMarried',
        'Married-AF-spouse': 'Married',
        'Married-civ-spouse': 'Married',
        'NotMarried': 'NotMarried',
        'Separated': 'Separated',
        'Divorced': 'Separated',
        'Widowed': 'Widowed',
    }
)
ordinal_columns={
    "education": ["dropout", "HighGrad", "CommunityCollege", "Bachelors", "Masters", "Doctorate"],
}
data["native-country"] = data["native-country"].replace(
{"Canada": "North America", "Cuba": "South America", "Dominican-Republic": "South America", "El-Salvador": "South America", "Guatemala": "South America",
                   "Haiti": "South America", "Honduras": "South America", "Jamaica": "South America", "Mexico": "South America", "Nicaragua": "South America",
                   "Outlying-US(Guam-USVI-etc)": "North America", "Puerto-Rico": "North America", "Trinadad&Tobago": "South America",
                   "United-States": "North America", "Cambodia": "Asia", "China": "Asia", "Hong": "Asia", "India": "Asia", "Iran": "Asia", "Japan": "Asia", "Laos": "Asia",
          "Philippines": "Asia", "Taiwan": "Asia", "Thailand": "Asia", "Vietnam": "Asia", "Columbia": "South America", "Ecuador": "South America", "Peru": "South America", "England": "West Europe", "France": "West Europe", "Germany": "West Europe", "Greece": "West Europe", "Holand-Netherlands": "West Europe",
            "Ireland": "West Europe", "Italy": "West Europe", "Portugal": "West Europe", "Scotland": "West Europe", "Hungary": "East Europe", "Poland": "East Europe", "Yugoslavia": "East Europe", "South": "?"})
data["capital"] = data["capital-gain"]-data["capital-loss"]
data.drop(["capital-gain", "capital-loss"], axis=1, inplace=True)
data["education-num"] = data["education-num"].astype(float)

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from automl.search.automl import AutoML

am = AutoML(
    random_state=42,
    level="common",
    target_metric="f1",
    trainer_config={
        #"secondary_level": ComponentLevel.UNCOMMON,
        "cache": True,
        "early_stopping": True,
        "return_test_scores_during_tuning": True,
        "tuning_time": 120*3,
        "stacking_level": 3,
        "tune_kwargs": {"max_concurrent": 2, "trainable_n_jobs": 4}
    },
    #cv=RepeatedStratifiedKFold(random_state=420, n_repeats=2, n_splits=4)
    cv=4,
)

In [ ]:
t_s = time()
am.fit(data.drop("income >50K", axis=1), data["income >50K"], ordinal_columns={"education":ordinal_columns})
#am.fit(data.drop("Purchase", axis=1), data["Purchase"])
#am.fit(data.drop("default", axis=1), data["default"], ordinal_columns={"EDUCATION": list(sorted(data["EDUCATION"].unique()))})
#am.fit(data.drop("quality", axis=1), data["quality"]>=6)
#am.fit(data.drop("target", axis=1), data["target"])
#am.fit(data.drop("target", axis=1), data["target"], ordinal_columns=ordinal_columns)
#am.fit(X_train, y_train, X_test=X_test, y_test=y_test)
t_end = time() - t_s

In [ ]:
am._get_results()

In [ ]:
a

In [ ]:
am.visualize_pipeline("349f7d86")

In [ ]:
x

In [ ]:
x.dtypes

In [ ]:
am.X_test_

In [ ]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

In [ ]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

In [ ]:
a.score(am.X_test_, am.y_test_)

In [ ]:
am.visualize_pipeline("65a71b94")

In [ ]:
am.results_.sort_values("Test accuracy", ascending=False)[:60]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

lr = LogisticRegression(random_state=42).fit(X_train, y_train)
dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
predictions = [lr.predict_proba(X_train), dt.predict_proba(X_train)]

In [ ]:
from automl_models.components.estimators.ensemble import PandasStackingClassifier
stack = PandasStackingClassifier(estimators=[("lr", lr), ("dt", dt)], final_estimator=LogisticRegression(random_state=42)).fit(X_train, y_train)

In [ ]:
p = stack.predict_proba(X_test)

In [ ]:
stack._saved_test_predictions=predictions
pp = stack.predict_proba(X_test)

In [ ]:
p == pp

In [ ]:
am.results_.loc["0_VotingUniform"]

In [ ]:
p = am.get_pipeline_by_id("0_VotingUniform")

In [ ]:
p.score(am.X_test_, am.y_test_)

In [ ]:
p._final_estimator

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from deslib.des import KNORAE, METADES, KNORAU
import numpy as np

In [ ]:
rfc = RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True)
X, y = load_breast_cancer(return_X_y=True)


In [ ]:
data = pd.read_csv("datasets/wine.csv")

In [ ]:
X = data.drop("type", axis=1).to_numpy()
y = LabelEncoder().fit_transform(data["type"].to_numpy())

In [ ]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

In [ ]:
class DESCV(ClassifierMixin, BaseEstimator):
    def __init__(self, des, classifier_pool, cv):
        self.des = des
        self.classifier_pool = classifier_pool
        self.cv = cv
    def fit(self, X, y):
        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        transformed_y = self.le_.transform(y)

        des = []
        for train_ind, test_ind in self.cv.split(X, y):
            rfc_ = clone(self.classifier_pool).fit(X[train_ind], y[train_ind])
            des_ = clone(self.des)
            des_.set_params(pool_classifiers=rfc_)
            des_.fit(X[test_ind], y[test_ind])
            des.append(des_)
        self.estimators_ = des

    def _predict(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([est.predict(X) for est in self.estimators_]).T

    def _collect_probas(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([clf.predict_proba(X) for clf in self.estimators_])

    def predict(self, X):
        maj = np.argmax(self.predict_proba(X), axis=1)
        maj = self.le_.inverse_transform(maj)

        return maj

    def predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting."""
        #check_is_fitted(self)
        avg = np.average(self._collect_probas(X), axis=0)
        return avg

In [ ]:
X, y = make_classification(n_samples=7000, random_state=42)

In [ ]:
descv = DESCV(KNORAE(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
desucv = DESCV(KNORAU(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
metadescv = DESCV(METADES(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
%%time
np.mean(cross_val_score(rfc, X, y, scoring="roc_auc_ovr_weighted"))

In [ ]:
np.mean(cross_val_score(rfc, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(descv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(descv, X, y, scoring="f1"))

In [ ]:
%%time
np.mean(cross_val_score(desucv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(desucv, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(metadescv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(metadescv, X, y, scoring="f1", error_score="raise"))

In [ ]:
rfc.fit(X, y)
rfc.oob_decision_function_

In [ ]:
np.argmax(rfc.oob_decision_function_, axis=1)

In [ ]:
rfc.oob_decision_function_.shape